<a href="https://colab.research.google.com/github/1st-award/andong_2021_2_1/blob/main/Python/%ED%95%84%EA%B8%B0%20%EC%82%AC%EB%B3%B8/202104koreanPopu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Choropleth map
- 지리적 영역 범휘별 수치 데이터 값을 **색**이나 패턴으로 표현한 지도유형
- 예: 미국의 대통령 선거에서 투표한 정당의 후보에 따라 각 주가 빨간색 또는 파란색으로 표시된 미국의 지도

## folium.map()의 tiles 속성
- OpenStreetMap: 기본
- Mapquestopen
- MapQuest Open Aerial
- Mapbox Bright
- Mapbox Control room
- **Stamen Terrain**
- **Stamen Toner**
- stamenwatercolor
- cartodbpositron
- cartodbdark_matter

## 데이터 가져오기 및 정제

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
import pandas as pd
import folium as fl
import json

In [ ]:
popuDF = pd.read_excel('/gdrive/My Drive/Python/202104_주민등록인구및세대현황.xlsx')
popuDF

In [ ]:
popuDF.drop([0, 1, 2], inplace=True)
popuDF.head()

In [ ]:
popuDF.drop(['※ 매월 말일자 통계 현황'], axis=1, inplace=True)
popuDF.head()

In [ ]:
# 행정기관 총인구수 세대수  세대당 인구 남자 인구수  여자 인구수  남여 비율
popuDF.columns = (['행정구역', '총인구수', '세대수', '세대당 인구', '남자', '여자', '남녀비율'])
popuDF.head()

In [ ]:
popuDF = popuDF.set_index('행정구역')
popuDF

In [ ]:
# 천 단위 구분 기호를 제거하고 문자열을 수로 변환
delComma = lambda x : x.replace(',','')
for c in range(5):
  popuDF.iloc[:, c] = popuDF.iloc[:, c].apply(delComma)

popuDF

In [ ]:
print('before', popuDF.dtypes)
for c in range(6):
  popuDF.iloc[:, c] = pd.to_numeric(popuDF.iloc[:, c])

print('after', popuDF.dtypes)

In [ ]:
# 시도청 공백 제거
delSpace = lambda x : x.replace(' ', '')
popuDF.index = map(delSpace, popuDF.index)
popuDF.index

In [ ]:
with open('/gdrive/My Drive/Python/TL_SCCO_CTPRVN.json') as json_file:
  korea_geo = json.load(json_file)
korea_geo

In [ ]:
# 각 광역시도청 위치 정보 파일 읽기
office = pd.read_csv("/gdrive/My Drive/Python/광역시도청_위치.csv", encoding='utf-8')
office.set_index(office['행정기관'], inplace = True)
del office['행정기관']
office

In [ ]:
popuKorea = fl.Map(location=[36, 127], tiles = 'stamenwatercolor', zoom_start=7)
fl.Choropleth(
    geo_data=korea_geo,
    name = 'Population Distribution by Region in Korea',
    data = popuDF['총인구수'],
    columns = [popuDF.index, popuDF['총인구수']],
    key_on = 'feature.properties.CTP_KOR_NM',
    fill_color='YlGnBu',
    fill_opacity = 0.7,
    line_opacitruety = 0.3,
    highlight = True,
    legend_name = '지역별 총인구수'
).add_to(popuKorea)

for i in range(len(office)):
  lati, longi = office.iat[i, 0], office.iat[i, 1]
  fl.Marker([lati, longi], tooltip = '[' + office.index[i] + '] ' + str(popuDF.iloc[i, 0])+ '명',
                icon=fl.Icon(icon='info-sign', color='red')).add_to(popuKorea)
popuKorea

## 전체 코드 정리

In [ ]:
# 전체 코드 정리
# 1. 라이브러리 가져오기
import pandas as pd
import folium as fl
import json
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# 2. 인구 데이터 파일 읽고 정제하기
popuDF = pd.read_excel('/gdrive/My Drive/Python/202104_주민등록인구및세대현황.xlsx')
popuDF.drop([0, 1, 2], inplace=True)
popuDF.drop(['※ 매월 말일자 통계 현황'], axis=1, inplace=True)
# 행정기관 총인구수 세대수  세대당 인구 남자 인구수  여자 인구수  남여 비율
popuDF.columns = (['행정구역', '총인구수', '세대수', '세대당 인구', '남자', '여자', '남녀비율'])
popuDF = popuDF.set_index('행정구역')
# 천 단위 구분 기호를 제거하고 문자열을 수로 변환
delComma = lambda x : x.replace(',','')
for c in range(5):
  popuDF.iloc[:, c] = popuDF.iloc[:, c].apply(delComma)
# 인구 데이터를 수로 변환
for c in range(6):
  popuDF.iloc[:, c] = pd.to_numeric(popuDF.iloc[:, c])
# 행정 구역명에 포함된 불필요한 공백 제거
delSpace = lambda x : x.replace(' ', '')
popuDF.index = map(delSpace, popuDF.index)
# 3. JSON 파일 읽기
with open('/gdrive/My Drive/Python/TL_SCCO_CTPRVN.json') as json_file:
  korea_geo = json.load(json_file)
# 4. 각 광역시도청 위치 정보 파일 읽기
office = pd.read_csv("/gdrive/My Drive/Python/광역시도청_위치.csv", encoding='utf-8')
office.set_index(office['행정기관'], inplace = True)
del office['행정기관']
office
# 5. 등치 지도 그리기
popuKorea = fl.Map(location=[36, 127], tiles = 'stamenwatercolor', zoom_start=7)
fl.Choropleth(
    geo_data=korea_geo,
    name = 'Population Distribution by Region in Korea',
    data = popuDF['총인구수'],
    columns = [popuDF.index, popuDF['총인구수']],
    key_on = 'feature.properties.CTP_KOR_NM',
    fill_color='YlGnBu',
    fill_opacity = 0.7,
    line_opacitruety = 0.3,
    highlight = True,
    legend_name = '지역별 총인구수'
).add_to(popuKorea)

# 6. 등치 지도 위에 각 시/도청 위치에 표식 달기
for i in range(len(office)):
  lati, longi = office.iat[i, 0], office.iat[i, 1]
  fl.Marker([lati, longi], tooltip = '[' + office.index[i] + '] ' + str(popuDF.iloc[i, 0])+ '명',
                icon=fl.Icon(icon='info-sign', color='red')).add_to(popuKorea)

# 지도 출력하기
popuKorea